In [1]:
from pathlib import Path


VIDEO = Path('../data/raw/e2vid/water_balloon_120fps.mp4')

In [2]:
import evs_explorer

scfg = evs_explorer.Configuration.from_yaml(
    simulator_config='../configs/data_generator/simulator/evs_explorer.yml',
    sensor_config='../configs/data_generator/simulator/davis_model.yml',
    sensor_model='davis_model'
)



scfg.input.source = VIDEO # TODO: better way of handling this?
threshold = 1.08
min_illuminance_lux = 650
max_illuminance_lux = 12500
scfg.input.desired_width = 640

scfg.optics.max_illuminance_lux = max_illuminance_lux
scfg.optics.min_illuminance_lux = min_illuminance_lux
scfg.sensor.ONth_mul = threshold
scfg.sensor.OFFth_mul = threshold



/home/davton/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading simulation config file from ../configs/data_generator/simulator/evs_explorer.yml
Loading sensor config file from ../configs/data_generator/simulator/davis_model.yml
GPU is available and will be used (cuda:0).
initializing pytorch / CUDA ... done


In [3]:
evs = evs_explorer.EvsExplorer(scfg)
events = evs.run("sensor_data")

Running EVS Explorer 2.0.0
Loading frames from: ../data/raw/e2vid/water_balloon_120fps.mp4
The given update noise frequency will allow only to see up to 10.0 Hz of noise, but it actually goes up to 147 Hz on average!
dumping config to output/config/water_balloon_120fps/sim_config.yml
dumping config to output/config/water_balloon_120fps/sensor_config.yml


 67%|██████▋   | 416/617 [00:10<00:04, 46.14it/s]

timing:
 - ConvertLuxToIrradiance: 0.00s = 0.04%
 - ConvertBGRtoGrayscale: 0.02s = 0.14%
 - StdoutProgressBar: 0.06s = 0.57%
 - Convert8bitToLux: 0.07s = 0.64%
 - FramesExtractor: 0.09s = 0.85%
 - ConvertToTorch: 0.37s = 3.48%
 - Downsample: 0.53s = 5.02%
 - SensorModelNode: 9.29s = 88.02%
All nodes: 10.42 seconds
Total runtime: 10.55 seconds
Total simulation time: 5.13 seconds
 -> simulation speed: 2.0550438955232693 seconds realtime per second of simulation
output: sensor_data


In [4]:
import numpy as np


bin_size = 0.02

number_of_temporal_bins = int(np.ceil((events.timestamp.max() / bin_size)))
max_t = number_of_temporal_bins * bin_size

In [5]:
from dynamic_fusion.data_generator.event_discretizer import EventDiscretizer
from dynamic_fusion.data_generator.configuration import EventDiscretizerConfiguration

config = EventDiscretizerConfiguration(number_of_temporal_bins=number_of_temporal_bins, number_of_temporal_sub_bins_per_bin=1)

discretizer = EventDiscretizer(config, max_timestamp=max_t)
size = (events.x.max() + 1, events.y.max() + 1 )

discretized_events = discretizer.run({1: events}, (size[1], size[0]))
discretized_events = discretized_events[1]

In [6]:
len(events) / events.timestamp.max() / (events.x.max() + 1) / (events.y.max() + 1)

15.64313216527433

In [7]:
from matplotlib import pyplot as plt
import numpy as np

from dynamic_fusion.utils.network import to_numpy
from dynamic_fusion.utils.visualization import create_red_blue_cmap, img_to_colormap
import cv2

size = (events.x.max() + 1, events.y.max() + 1 )

out = cv2.VideoWriter("simulated_water_1.08.mp4", cv2.VideoWriter.fourcc(*"mp4v"), 10, size)

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)  # White color
line_type = 2
position = (10, 50)  # Position of the text (bottom left corner)


for i, eps in enumerate(discretized_events.event_polarity_sum):
    colored_event_polarity_sum = img_to_colormap(to_numpy(eps.sum(dim=0)), create_red_blue_cmap(501))

    evr = discretized_events.event_count[i].sum(dim=0).mean() * number_of_temporal_bins / max_t

    frame_processed = (colored_event_polarity_sum[:, ::-1] * 255).astype(np.uint8)


    cv2.putText(frame_processed, f"Events per second per pixel={evr:.2f}", position, font, font_scale, font_color, line_type)

    out.write(frame_processed)
out.release()


 68%|██████▊   | 417/617 [00:29<00:04, 46.14it/s]